In [1]:
import os # работа с папкой и файлами
import re # регулярные выражения, доп вариант к очистке
import string # работа со строкой

import pandas as pd # работа с таблицей dataframe

import nltk # работа с пакетами языков
from nltk import word_tokenize, ngrams # токенизация и деление на n граммы
from nltk.corpus import stopwords # стопслова, extend

from wordcloud import WordCloud # визуальное отображение
import pymorphy3 # работа с русским языком, pymorphy3

import matplotlib.pyplot as plt # визуальное отображение

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
# Список файлов в папке
FOLDER_NAME = 'data'

direct_list = os.listdir(FOLDER_NAME)
direct_list

['Ария - Возьми мое сердце.txt',
 'Григорий Лепс - Самый лучший день.txt',
 'Земляне - Земля в иллюминаторе.txt',
 'Кино - Пачка сигарет.txt',
 'Кипелов - Я Свободен.txt',
 'Король и Шут - Лесник.txt',
 'Любэ - Берёзы.txt',
 'Любэ - Ты неси меня река.txt',
 'Наутилус Помпилиус - Прогулки по воде.txt',
 'Руки Вверх - Крошка моя.txt']

In [7]:
# Имена песен из названия файлов
names = [song.replace(".txt", "") for song in direct_list]
names

['Ария - Возьми мое сердце',
 'Григорий Лепс - Самый лучший день',
 'Земляне - Земля в иллюминаторе',
 'Кино - Пачка сигарет',
 'Кипелов - Я Свободен',
 'Король и Шут - Лесник',
 'Любэ - Берёзы',
 'Любэ - Ты неси меня река',
 'Наутилус Помпилиус - Прогулки по воде',
 'Руки Вверх - Крошка моя']

In [6]:
# Функция чтения файла
def read_song(filename, folder=FOLDER_NAME):
  text = ''
  with open(f'{folder}/{filename}.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\n', ' ')
  return text

# Чтение данных из папки
songs_list = []

for song in names:
  songs_list.append(read_song(song))

songs_list

['Слепая ночь легла у ног И не пускает на порог. Брожу по дому как во сне, Но мне покоя нет нигде. Тупая боль пробьет висок, И пальцы лягут на курок, А в зеркалах качнется призрак, Призрак любви...  Возьми мое сердце, Возьми мою душу. Я так одинок в этот час, Что хочу умереть... Мне некуда деться, Свой мир я разрушил. По мне плачет только свеча, На холодной заре.  Ты умерла в дождливый день И тени плыли по воде Я смерть увидел в первый раз Ее величие и грязь В твоих глазах застыла боль, Я разделю ее с тобой. А в зеркалах качнется призрак, Призрак любви...  Возьми мое сердце, Возьми мою душу. Я так одинок в этот час, Что хочу умереть... Мне некуда деться, Свой мир я разрушил. По мне плачет только свеча, На холодной заре.  Я слышу утренний колокол - Он славит праздник, И сыпет медью и золотом. Ты теперь в царстве вечного сна.  Я слышу утренний колокол - Он бесов дразнит, И звоном небо расколото. На земле я любил лишь тебя...  Я слышу утренний колокол - Он славит праздник, И сыпет медью и

In [5]:
# Стоп-слова русского языка
stopwords_list = stopwords.words("russian")

# Собственные стоп-слова
extra_stop = ['твоей', 'свои', 'тобой', 'мой', 'твой', 'мой', 'это']
stopwords_list.extend(extra_stop)

len(stopwords_list)

158